In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/Appliance-Energy-Prediction")
import pandas as pd
from pca_fts.PcaSarimax import PcaSarimax
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
import math

In [2]:
def sample_first_prows(data, perc=0.75):
    return data.head(int(len(data)*(perc)))

In [3]:
filename = '/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/energydata_complete.csv'
data = pd.read_csv(filename)
data.pop('date')
data.pop('rv1')
data.pop('rv2')
data

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,...,18.2000,48.900000,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000
1,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,...,18.2000,48.863333,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000
2,50,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,...,18.2000,48.730000,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000
3,50,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,...,18.1000,48.590000,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000
4,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,...,18.1000,48.590000,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,...,24.7000,50.074000,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333
19731,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,...,24.7000,49.790000,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000
19732,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,...,24.7000,49.660000,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667
19733,420,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,...,24.6625,49.518750,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333


In [4]:
# diff = data.loc[:,'lights':'Tdewpoint']
# diff = diff.diff()
# diff['Appliances'] = data['Appliances']
# data = diff.dropna()

In [5]:
pca_sarimax = PcaSarimax(n_components = 2,
                       endogen_variable = 'Appliances',
                       order = [3, 1, 2])

In [6]:
reduced = pca_sarimax.apply_pca(data)
reduced

,x,y,Appliances
0,-0.719141,0.591053,60
1,-0.718122,0.586814,60
2,-0.718488,0.571549,50
3,-0.724129,0.558693,50
4,-0.727253,0.555068,60
...,...,...,...
19730,1.375161,0.715077,100
19731,1.365770,0.708007,90
19732,1.352865,0.716462,270
19733,1.333627,0.706385,420


In [7]:
exog = reduced.drop(labels=['Appliances'], axis=1)
exog

,x,y
0,-0.719141,0.591053
1,-0.718122,0.586814
2,-0.718488,0.571549
3,-0.724129,0.558693
4,-0.727253,0.555068
...,...,...
19730,1.375161,0.715077
19731,1.365770,0.708007
19732,1.352865,0.716462
19733,1.333627,0.706385


In [8]:
endog = reduced['Appliances']
endog

0         60
1         60
2         50
3         50
4         60
        ... 
19730    100
19731     90
19732    270
19733    420
19734    430
Name: Appliances, Length: 19735, dtype: int64

In [9]:
# from pmdarima.arima import auto_arima
# step_wise=auto_arima(endog, 
#  exogenous= exog,
#  start_p=1, start_q=1, 
#  max_p=4, max_q=4, 
#  d=1, max_d=4,
#  trace=True, 
#  error_action='ignore', 
#  suppress_warnings=True, 
#  stepwise=True)

In [10]:
pca_sarimax = PcaSarimax(n_components = 2,
                       endogen_variable = 'Appliances',
                       order = [3,1,2])

In [11]:
train = sample_first_prows(data,0.75)
test = data.iloc[max(train.index):]
y_test = data.iloc[max(train.index):]['Appliances'].values

In [12]:
model, sarimax, pca_reduced_train = pca_sarimax.run_train_model(train)

/home/hugo/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [13]:
sarimax.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                14801
Model:               SARIMAX(3, 1, 2)   Log Likelihood              -83646.825
Date:                Wed, 16 Jun 2021   AIC                         167355.649
Time:                        16:53:06   BIC                         167591.317
Sample:                             0   HQIC                        167433.893
                              - 14801                                         
Covariance Type:                  opg                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
lights          0.5208      0.080      6.514      0.000       0.364       0.678
T1             -6.6499      9.246     -0.719      0.472     -24.772      11.473
RH_1          -22.9018      0.668    -34.288      0.000     -24.211     -21.593
T2             25.3547      5.809      4.364      0.000      13.969      36.741
RH_2           -2.5996      1.409     -1.845      0.065      -5.362       0.163
T3            -95.9714      6.085    -15.772      0.000    -107.898     -84.045
RH_3          -15.8578      1.923     -8.247      0.000     -19.626     -12.089
T4            -47.3086      3.890    -12.162      0.000     -54.933     -39.684
RH_4            8.3398      2.238      3.727      0.000       3.954      12.725
T5              6.1505      5.168      1.190      0.234      -3.978      16.279
RH_5            0.1383      0.282      0.491      0.623      -0.414       0.690
T6              0.7077      2.516      0.281      0.778      -4.223       5.639
RH_6            0.6414      0.379      1.694      0.090      -0.101       1.384
T7              6.3085      8.636      0.730      0.465     -10.618      23.236
RH_7            0.8498      2.252      0.377      0.706      -3.565       5.265
T8              9.2682      7.234      1.281      0.200      -4.911      23.447
RH_8            7.5525      2.161      3.495      0.000       3.317      11.788
T9             15.6765     11.559      1.356      0.175      -6.979      38.332
RH_9           -8.7504      2.065     -4.237      0.000     -12.798      -4.703
T_out          -1.5320     11.655     -0.131      0.895     -24.375      21.311
Press_mm_hg    -1.1472      1.846     -0.621      0.534      -4.766       2.471
RH_out         -2.3066      2.268     -1.017      0.309      -6.752       2.138
Windspeed      -3.5765      2.013     -1.777      0.076      -7.521       0.368
Visibility      0.8496      0.194      4.378      0.000       0.469       1.230
Tdewpoint       7.6625     11.261      0.680      0.496     -14.408      29.733
ar.L1           1.4784      0.020     75.489      0.000       1.440       1.517
ar.L2          -0.5982      0.016    -36.592      0.000      -0.630      -0.566
ar.L3           0.0968      0.006     16.589      0.000       0.085       0.108
ma.L1          -1.6690      0.019    -85.612      0.000      -1.707      -1.631
ma.L2           0.6712      0.019     34.919      0.000       0.634       0.709
sigma2       4734.3245     19.809    239.003      0.000    4695.500    4773.149
===================================================================================
Ljung-Box (L1) (Q):                  14.05   Jarque-Bera (JB):            261069.94
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.85   Skew:                             2.87
Prob(H) (two-sided):                  0.00   Kurtosis:                        22.76
===================================================================================

Warnings:
[1] Co

In [14]:
pca_reduced_train

,x,y,Appliances
0,-0.790709,0.386894,60
1,-0.788998,0.379760,60
2,-0.785144,0.363308,50
3,-0.787687,0.346883,50
4,-0.787631,0.338269,60
...,...,...,...
14796,0.947253,0.023656,370
14797,0.958029,0.013807,400
14798,0.964542,0.007168,300
14799,0.983955,0.004362,300


In [15]:
start = len(train)
end = len(train) + len(test) -1
forecast, pca_reduced_test = pca_sarimax.run_test_model(test, sarimax,start,end)

In [16]:
list(forecast)

[263.7637404187135,
 242.94354767021014,
 195.09827929847506,
 152.9880932378237,
 118.29169426890076,
 65.51292121651022,
 66.41763377738516,
 77.15004741196844,
 105.52815852975436,
 129.8211425072559,
 149.15427175024797,
 164.17665527006557,
 173.5846970074008,
 170.38599719329613,
 166.0121831164397,
 166.50359187087815,
 163.00763690988333,
 178.31388697997954,
 189.14615915232662,
 192.21195175604043,
 204.854157054986,
 209.74120770409718,
 220.15341063559208,
 234.47953241108462,
 238.61703362982917,
 236.510648064027,
 248.43271542361708,
 240.66311336517174,
 250.79057008732434,
 250.3997157853173,
 266.72330924381276,
 265.1669663968387,
 268.76302090069385,
 282.7694229413255,
 287.0620696766964,
 292.8258459273384,
 289.0378078045351,
 297.3667455115901,
 302.25033097189043,
 305.8832952566099,
 309.44895901773907,
 267.59521084711014,
 206.46130228364473,
 167.52299860240055,
 187.54109579082888,
 178.14608496512574,
 191.9299030176535,
 197.14939737305303,
 215.05918059

In [17]:
pca_reduced_test['Appliances']

0       280
1       280
2       320
3       360
4       210
       ... 
4930    100
4931     90
4932    270
4933    420
4934    430
Name: Appliances, Length: 4935, dtype: int64

In [18]:
print(Measures.rmse(pca_reduced_test['Appliances'],forecast.values))

267.95379675319657


In [19]:
print(Measures.mape(pca_reduced_test['Appliances'],forecast.values))

320.32313924809404


In [20]:
pca_sarimax = PcaSarimax(n_components = 2,
                       endogen_variable = 'Appliances',
                       order=[3,1,2])

In [21]:
result = {
     "window": [],
     "rmse": [],
     "mape": [],
     "smape": []
}

tam = len(data)
n_windows = 30
windows_length = math.floor(tam / n_windows)
for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
    if len(ttest) > 0:
        
        start = len(ttrain)
        end = len(ttrain)+ len(ttest) -1
        appliance = ttest['Appliances'].reset_index()
        
        print('-' * 20)
        print(f'training window {(ct)}')
        model, sarimax, pca_reduced_train = pca_sarimax.run_train_model(ttrain)
        forecast, pca_reduced_test = pca_sarimax.run_test_model(ttest,sarimax,start,end)
        
        print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
        rmse = Measures.rmse(list(appliance.iloc[:,1]),list(forecast))
        mape = Measures.mape(list(appliance.iloc[:,1]),list(forecast))
        smape = Measures.smape(list(appliance.iloc[:,1]),list(forecast))
        
        result["rmse"].append(rmse)
        result["mape"].append(mape)
        result["smape"].append(smape)
        result["window"].append(ct)
        
measures = pd.DataFrame(result)

--------------------
training window 0


/home/hugo/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


[ 16:53:11] getting statistics
--------------------
training window 657


/home/hugo/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


[ 16:53:17] getting statistics
--------------------
training window 1314


...

/home/hugo/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


[ 16:56:09] getting statistics
--------------------
training window 18396


/home/hugo/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


[ 16:56:17] getting statistics
--------------------
training window 19053
[ 16:56:22] getting statistics


/home/hugo/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [25]:
measures

,window,rmse,mape,smape
0,0,363.906489,394.999380,84.292867
1,657,867.197179,1147.167234,73.762855
2,1314,641.206406,802.871934,95.062856
3,1971,329.066879,880.914686,76.390941
4,2628,127.167824,114.349253,39.924320
5,3285,143.614262,215.199782,54.080303
6,3942,211.878657,239.673592,49.947262
7,4599,134.510687,180.873834,40.925091
8,5256,193.200121,310.449932,63.296028
9,5913,151.273417,74.690501,54.213116


In [26]:
print("Mean: ")
print(statistics.mean(measures['rmse']))
print(statistics.mean(measures['mape']))
print(statistics.mean(measures['smape']))

Mean: 
320.0932473048628
417.9062718023049
66.74305529534749


In [27]:
# appliance = test['Appliances'].reset_index()
# list(appliance.iloc[:,1])